In [53]:
from datasets import load_titanic, balance_classes
from tools import model_performance,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_roc_curve
from pygam import LogisticGAM, s, f, l
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
df, df_train, df_test = load_titanic()
df_train.columns

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'target'], dtype='object')

In [3]:
original_X = df.drop(['target'], axis=1)
original_y = df['target']
train_X = df_train.drop(['target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['target'], axis=1)
test_y = df_test['target']

In [4]:
original_X = pd.get_dummies(original_X)
train_X = pd.get_dummies(train_X)
test_X = pd.get_dummies(test_X)

features = train_X.columns

In [5]:
scaler = StandardScaler().fit(train_X)
original_X = scaler.transform(original_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

## Pocet komponent

In [30]:
pca = PCA(n_components=len(features)-1)
pca = pca.fit(original_X)

In [31]:
pca.explained_variance_ratio_

array([0.26164337, 0.20496143, 0.15260618, 0.13179013, 0.07820655,
       0.07017069, 0.06428829, 0.03633335, 0.        ])

## Vizualizace

In [34]:
pca = PCA(n_components=3)
pca = pca.fit(original_X)

In [35]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [36]:
transformed_df = pd.DataFrame(transformed_original_X, columns=['x','y','z'])
transformed_df['target'] = original_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target')

## Classification

In [44]:
pca = PCA(n_components=4)
pca = pca.fit(original_X)

In [45]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [60]:
model = LogisticRegression(penalty='none')

In [62]:
# stratified (balanced), 5-fold - zadne zlepseni oproti ciste LogisticRegression
cross_val_score(model, transformed_original_X, original_y).mean()

0.7856380641516539

In [63]:
model = model.fit(transformed_train_X, train_y)

In [64]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.848314606741573
Test score
0.7601683029453016


In [ ]:
confusion_matrix(model.predict(transformed_train_X), train_y, model.predict(transformed_test_X), test_y)